## **Article Retrieval System using FAISS and GPT-3.5-turbo**

Installing necessary libraries for the project

In [1]:
%pip install ydata-profiling langchain sentence_transformers faiss-cpu openai python-dotenv setuptools

In [2]:
import pandas as pd
import torch
from openai import OpenAI
import os
from dotenv import load_dotenv
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from ydata_profiling import ProfileReport
import warnings

In [3]:
warnings.filterwarnings("ignore")

Loading the environment variables from the config.env file, this file contains OpenAI API key

In [4]:
load_dotenv('config.env')
openai_client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

Path to the CSV file containing the dataset

In [5]:
filename = 'medium.csv'

Model name used for embeddings

In [6]:
model_name="BAAI/bge-base-en-v1.5"

Check for GPU availability and set up the device accordingly

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device.type == 'cuda':
    gpu_name = torch.cuda.get_device_name(0)
    total_memory = torch.cuda.get_device_properties(0).total_memory
    total_memory_gb = total_memory / (1024**3)
    print(f"GPU is available: {gpu_name} with {total_memory_gb:.2f} GB")
else:
    print("GPU is not available. Using CPU")

GPU is not available. Using CPU


Load the dataset into a Pandas DataFrame and generate a profile report

In [8]:
df = pd.read_csv(filename)
profile = ProfileReport(df, title='1300 Kaggle Articles about Data Science')
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Load the articles using the DataFrameLoader and split the documents into chunks

In [9]:
articles = DataFrameLoader(df, page_content_column="Title")
document = articles.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=30)
splitted_texts = splitter.split_documents(document)

Index the split documents using FAISS for efficient similarity search

In [10]:
db = FAISS.from_documents(splitted_texts, HuggingFaceEmbeddings(model_name=model_name))

Function to display retrieved documents based on a query, the size of displayed docs is limited

In [11]:
def show_rag(query):
    # Retrieve similar documents using FAISS
    docs = db.similarity_search(query, k=3)
    print(f'Query: {query}')
    print(f'Retrieved documents: {len(docs)}')
    # Iterate over and display each retrieved document
    for index, doc in enumerate(docs, start=1):
        details = doc.to_json()['kwargs']
        title = details['page_content']
        text = details['metadata']['Text'][:500]
        print(f"Document {index}: {title}\n")
        print(f"Text: {text}.. \n")
        print('-' * 80)

Example query to test document retrieval

In [12]:
query_text = "What is kNN?"
show_rag(query_text)

Query: What is kNN?
Retrieved documents: 3
Document 1: Layman’s Introduction to KNN

Text: Layman’s Introduction to KNN

Photo by timJ on Unsplash

kNN stands for k-Nearest Neighbours. It is a supervised learning algorithm. This means that we train it under supervision. We train it using the labelled data already available to us. Given a labelled dataset consisting of observations (x,y), we would like to capture the relationship between x — the data and y — the label. More formally, we want to learn a function g : X→Y so that given an unseen observation X, g(x) can confidently pre.. 

--------------------------------------------------------------------------------
Document 2: K-Nearest Neighbors (KNN) Algorithm

Text: K-Nearest Neighbors (KNN) Algorithm

A Brief Introduction Afroz Chakure · Follow Published in DataDrivenInvestor · 4 min read · Jul 6, 2019 -- Listen Share

Simple Analogy for K-Nearest Neighbors (K-NN)

In this blog, we’ll talk about one of the most widely used machine l

Function to retrieve documents and prepare them for the LLM

In [13]:
def retrieve_documents(query, k=3):
    # Perform a search to find relevant documents
    docs = db.similarity_search(query, k=k)
    retrieved_texts = [doc.to_json()['kwargs']['metadata']['Text'] for doc in docs]
    return retrieved_texts

Function to format documents into a prompt for GPT-3.5-turbo, the limit of tokens from document is limited here

In [14]:
def format_documents_for_prompt(retrieved_documents):
    # Join the document texts
    formatted_docs = "\n\n".join([f"Document: {doc[:500]}" for doc in retrieved_documents])
    return formatted_docs

Function to use GPT-3.5-turbo model for generating responses

In [15]:
def show_llm(query, retrieved_documents, max_tokens, model="gpt-3.5-turbo"):
    # Format the documents and craft the prompt for the LLM
    information = format_documents_for_prompt(retrieved_documents)
    # Query the OpenAI API with the prompt
    response = openai_client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant. You will be shown the user's question and the relevant information from the articles.  Answer the user's question using only this information."
            },
            {
                "role": "user",
                "content": f"Question: {query}. \n Information: {information}"
            }
        ],
        # max_tokens=max_tokens
    )
    # Return the generated response
    return response.choices[0].message.content

Example usage of the LLM to generate a response, there is an option to limit the output from model, to do this uncomment the part of code in llm()

In [16]:
query_text = "What is kNN?"
retrieved_documents = retrieve_documents(query_text)
max_tokens = 150
show_llm(query=query_text, retrieved_documents=retrieved_documents, max_tokens = max_tokens, model="gpt-3.5-turbo")

'kNN stands for k-Nearest Neighbours. It is a supervised learning algorithm where we train it using labelled data already available to us. The goal is to learn a function that can predict the label of unseen observations based on the relationship between the data and the label in the labelled dataset.'